In [18]:
num_samples = 3
interval = 33.0e-3
num_frames = 100
min_sigma = 1
max_sigma = 4
threshold = 50.0
overlap = 0.5

/home/manabu/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
nproc = 8

In [20]:
import numpy
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)

In [21]:
import mlflow

In [22]:
mlflow.start_run(run_name="analysis1")

<ActiveRun: >

In [24]:
mlflow.tracking.MlflowClient().get_run("0caadb6d6a4c49af977c7bdb14212bd9")

<Run: data=<RunData: metrics={}, params={'exposure_time': '0.033', 'seed': '123'}, tags={'mlflow.source.name': '/home/manabu/miniconda3/lib/python3.8/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'manabu'}>, info=<RunInfo: artifact_uri='./mlruns/0/0caadb6d6a4c49af977c7bdb14212bd9/artifacts', end_time=1601277112635, experiment_id='0', lifecycle_stage='active', run_id='0caadb6d6a4c49af977c7bdb14212bd9', run_uuid='0caadb6d6a4c49af977c7bdb14212bd9', start_time=1601274758213, status='FINISHED', user_id='manabu'>>

In [25]:
info=mlflow.tracking.MlflowClient().get_run("0caadb6d6a4c49af977c7bdb14212bd9").info

In [26]:
info.artifact_uri

'./mlruns/0/0caadb6d6a4c49af977c7bdb14212bd9/artifacts'

In [29]:
import pathlib
inputpath = pathlib.Path(info.artifact_uri)
artifacts = pathlib.Path(info.artifact_uri)
import scopyon

/home/manabu/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

for i in range(num_samples):
    imgs = [scopyon.Image(data) for data in numpy.load(inputpath / f"images{i:03d}.npy")]
    spots = [
        scopyon.analysis.spot_detection(
            img.as_array(), processes=nproc,
            min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold, overlap=overlap)
        for img in imgs]

    spots_ = []
    for t, data in zip(timepoints, spots):
        spots_.extend(([t] + list(row) for row in data))
    spots_ = numpy.array(spots_)
    numpy.save(artifacts / f"spots{i:03d}.npy", spots_)
    
    print("{} spots are detected in {} frames.".format(len(spots_), len(imgs)))

warnings.resetwarnings()

/home/manabu/miniconda3/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)
/home/manabu/miniconda3/lib/python3.8/site-packages/skimage/util/dtype.py:226: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  dtypeobj_out = np.dtype(dtype)


886 spots are detected in 3 frames.
867 spots are detected in 3 frames.
854 spots are detected in 3 frames.


In [31]:
mlflow.end_run()

/home/manabu/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
